In [1]:
# This is an early algo version #1 determining the best investnent window
# 2/18/2025
# Next steps:
#   1. Return more than one investment window
#   2. Implementation of the asset class policy
#   3. Implementation of the amounts that must mature

import mysql.connector
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv(verbose=True)

# Database connection parameters
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME"),
}

print('Database URL: ' + os.environ.get('DB_HOST'))

Database URL: timetables.mysql.database.azure.com


In [2]:
# Fetch data from database and return as a DataFrame
def fetch_data(table_name, column_names='*', condition='1'):
    try:
        # Connect
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        # Fetch
        query = f"SELECT {column_names} FROM {table_name} WHERE {condition}"
        cursor.execute(query)
        # Fetch column names
        columns = [col[0] for col in cursor.description]
        # Fetch data
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns)
        return df
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()

In [3]:
# Query the running balance of the asset classes
table_name = 'assetclassbalances'
column_names = 'TransactionDate, Portfolio, Cash_Sweep, US_Treasuries, CDs, CommercialPapers, MoneyMarket, MutualFund, US_Agencies'
condition = 'Title '
assets = fetch_data(table_name, column_names)

assets


,TransactionDate,Portfolio,Cash_Sweep,US_Treasuries,CDs,CommercialPapers,MoneyMarket,MutualFund,US_Agencies
0,2025-01-21,654628445.37,151070374.54,163716733.42,2600000.00,170669204.47,20065580.67,18975581.76,127530970.51
1,2025-01-22,492520445.37,37442374.54,163716733.42,2600000.00,156169204.47,20065580.67,18975581.76,93550970.51
2,2025-01-23,492520445.37,47442374.54,163716733.42,2600000.00,146169204.47,20065580.67,18975581.76,93550970.51
3,2025-01-24,470088445.37,33510374.54,163716733.42,2600000.00,137669204.47,20065580.67,18975581.76,93550970.51
4,2025-01-25,470088445.37,33510374.54,163716733.42,2600000.00,137669204.47,20065580.67,18975581.76,93550970.51
...,...,...,...,...,...,...,...,...,...
156,2025-06-26,414083445.37,258505374.54,-1883266.58,2600000.00,103669204.47,20065580.67,18975581.76,12150970.51
157,2025-06-27,412783445.37,257205374.54,-1883266.58,2600000.00,103669204.47,20065580.67,18975581.76,12150970.51
158,2025-06-28,412783445.37,257205374.54,-1883266.58,2600000.00,103669204.47,20065580.67,18975581.76,12150970.51
159,2025-06-29,412783445.37,257205374.54,-1883266.58,2600000.00,103669204.47,20065580.67,18975581.76,12150970.51


In [4]:
# Query the asset class max. percentages
table_name = 'AssetClass'
column_names = 'Title AS "AssetClass", PercentMax'
condition = 'AssetClassParentID = 0 && Title != "Not Assigned"'
max_percent = fetch_data(table_name, column_names, condition)
if max_percent is not None:
    # Insert an asset class key for column mapping
    asset_class_mapping = {
        'Cash/Sweep': 'Cash_Sweep',
        'US Treasuries': 'US_Treasuries',
        'Certificate of Deposit': 'CDs',
        'Commercial Paper': 'CommercialPapers',
        'Money Market': 'MoneyMarket',
        'Mutual Fund': 'MutualFund',
        'US Agencies': 'US_Agencies'
    }
    max_percent['AssetClassKey'] = max_percent['AssetClass'].map(asset_class_mapping)

    # Convert PercentMax to float
max_percent['PercentMax'] = max_percent['PercentMax'].astype(float)

print(max_percent)
# max_percent.info()

               AssetClass  PercentMax     AssetClassKey
0              Cash/Sweep       1.000        Cash_Sweep
1  Certificate of Deposit       0.500               CDs
2        Commercial Paper       0.333  CommercialPapers
3             US Agencies       0.200       US_Agencies
4            Money Market       0.200       MoneyMarket
5             Mutual Fund       0.200        MutualFund
6           US Treasuries       1.000     US_Treasuries


In [5]:
# Get the available cash min and max values
min_cash = assets['Cash_Sweep'].min()
max_cash = assets['Cash_Sweep'].max()
print(f"Min:{min_cash:,}" + '  ' f"Max:{max_cash:,}")

# Create a df with values between min_cash and max_cash (inclusive) named 'Cash'.
def prorate_cash_values(num_values, min_cash, max_cash):
    prorated_values = pd.Series(np.linspace(min_cash, max_cash, num_values))
    prorated_df = pd.DataFrame({'Cash': prorated_values})
    prorated_df['Cash'] = prorated_df['Cash'].astype(int)
    return prorated_df

# Prorate the cash interval values
min_investment = 10000
num_values = int((max_cash - min_cash) / min_investment)
print(num_values)

prorated_df = prorate_cash_values(num_values, min_cash, max_cash)
prorated_df


Min:28,568,374.54  Max:387,489,374.54
35892


,Cash
0,28568374
1,28578374
2,28588375
3,28598375
4,28608375
...,...
35887,387449373
35888,387459373
35889,387469373
35890,387479374


In [ ]:
# Reverse test (check) the increment delta by subtracting the second cash value from the first
#     Q: Why is there a difference?
increment = prorated_df['Cash'].iloc[1] - prorated_df['Cash'].iloc[0]
f"For {num_values:,} prorated values the increment is: ${increment:,}"

'For 35,892 prorated values the increment is: $10,000'

In [ ]:
# For each prorated cash value, compare it with the available cash_sweep value per day (True|False matrix)

# base dataframe
algo_df = assets[['TransactionDate', 'Cash_Sweep']].sort_values(by='TransactionDate', ascending=True).rename(columns={'TransactionDate': 'Date', 'Cash_Sweep': 'Cash'})

# matrix of comparisons (all at once)
cash_values = prorated_df['Cash'].values
cash_matrix = algo_df['Cash'].astype(float).values[:, np.newaxis] >= cash_values
column_names = [f'Less_{val}' for val in cash_values]

# result DataFrame
result_df = pd.DataFrame(cash_matrix, columns=column_names, index=algo_df.index)
# Combine the matix with the original dataframe
algo_df = pd.concat([algo_df, result_df], axis=1)

algo_df

,Date,Cash,Less_28568374,Less_28578374,Less_28588375,Less_28598375,Less_28608375,Less_28618376,Less_28628376,Less_28638376,...,Less_387399371,Less_387409372,Less_387419372,Less_387429372,Less_387439373,Less_387449373,Less_387459373,Less_387469373,Less_387479374,Less_387489374
0,2025-01-21,151070374.54,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,2025-01-22,37442374.54,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,2025-01-23,47442374.54,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,2025-01-24,33510374.54,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
4,2025-01-25,33510374.54,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,2025-06-26,258505374.54,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
157,2025-06-27,257205374.54,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
158,2025-06-28,257205374.54,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
159,2025-06-29,257205374.54,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [18]:
# Count the max. number of vertically consecutive True values in each column

# Count consecutive True values
def count_consecutive_true(series):
    # Convert boolean series to integers (True -> 1, False -> 0)
    int_series = series.astype(int)
    # Create groups when values change
    groups = (int_series != int_series.shift()).cumsum()
    # Count consecutive values in each group
    consecutive_counts = int_series.groupby(groups).cumsum()
    # Return maximum count
    return consecutive_counts.max() if len(consecutive_counts) > 0 else 0

# Apply the function to each boolean column (skip Date and Cash columns) and store in a series
max_consecutive_days = algo_df.iloc[:, 2:].apply(count_consecutive_true)
print("Maximum consecutive True values for each threshold:")

max_consecutive_days


Maximum consecutive True values for each threshold:


Less_28568374     161
Less_28578374     154
Less_28588375     154
Less_28598375     154
Less_28608375     154
                 ... 
Less_387449373      5
Less_387459373      5
Less_387469373      5
Less_387479374      5
Less_387489374      5
Length: 35892, dtype: int64

In [ ]:
# Convert the series to a DataFrame and store the max. consecutive days count and also compute the return
max_consecutive_days_df = max_consecutive_days.reset_index()

# Drop the prefic "Less_" from the index column values and convert it to an integer
max_consecutive_days_df['index'] = max_consecutive_days_df['index'].str.replace('Less_', '').astype(int)

# Rename the columns
max_consecutive_days_df.columns = ['Cash', 'Days']

# Multiply both columns and store the result in a third column "Return"
max_consecutive_days_df['Return'] = max_consecutive_days_df['Cash'] * max_consecutive_days_df['Days']


In [ ]:
#sort by Return and return the top x rows
max_consecutive_days_df.sort_values(by='Return', ascending=False).head(10)

#TODO: Get the top x day counts (unique values in the Days column)
#TODO: Get the amount and the consecutive TRUEs (days) instead of the max consecutive days count
#TODO: Look up the rows in the algo_df that match the top x day counts
#TODO: The start and end dates for each window instead of the following max. consecutive days


,Cash,Days,Return
7296,101530610,123,12488265030
7295,101520610,123,12487035030
7294,101510610,123,12485805030
7293,101500609,123,12484574907
7292,101490609,123,12483344907
7291,101480609,123,12482114907
7290,101470608,123,12480884784
7289,101460608,123,12479654784
7288,101450608,123,12478424784
7287,101440607,123,12477194661


In [ ]:
# Get the Cash, Days, and Return (row) with maximum return
max_return_row = max_consecutive_days_df.loc[max_consecutive_days_df['Return'].idxmax()]
print(max_return_row)

isolated_result_df = algo_df[[f"Less_{max_return_row['Cash']}", 'Date', 'Cash']]
isolated_result_df

Cash        101530610
Days              123
Return    12488265030
Name: 7296, dtype: int64


,Less_101530610,Date,Cash
0,True,2025-01-21,151070374.54
1,False,2025-01-22,37442374.54
2,False,2025-01-23,47442374.54
3,False,2025-01-24,33510374.54
4,False,2025-01-25,33510374.54
...,...,...,...
156,True,2025-06-26,258505374.54
157,True,2025-06-27,257205374.54
158,True,2025-06-28,257205374.54
159,True,2025-06-29,257205374.54


In [ ]:
# Finally using the count_consecutive_true function, find the maximum number of consecutive days
# and return the start and end dates and also the max. Cash value

def find_consecutive_dates(series):
    # Convert to boolean series
    bool_series = series.astype(bool)
    # Find where values change
    changes = bool_series != bool_series.shift()
    # Create groups
    groups = changes.cumsum()

    # Find lengths of True sequences
    lengths = bool_series.groupby(groups).sum()
    # Find the group with maximum consecutive True values
    max_group = lengths[lengths == lengths.max()].index[0]

    # Get dates where this group starts and ends
    dates = series.index[groups == max_group]
    start_date_index = dates[0]
    end_date_index = dates[-1]

    start_date = algo_df.loc[start_date_index, 'Date']
    end_date = algo_df.loc[end_date_index, 'Date']

    return start_date, end_date, lengths.max()

# Get column name with max consecutive days
col = f"Less_{max_return_row['Cash']}"

# Find start date, end date and max consecutive days
start_date, end_date, max_days = find_consecutive_dates(algo_df[col])

print("Max. ROI investment window:")
print(f"Start date: {start_date}")
print(f"End date: {end_date}")
print(f"Length: {max_days} days")
print(f"Available cash: ${int(max_return_row['Cash']):,}")



Max. ROI investment window:
Start date: 2025-02-28
End date: 2025-06-30
Length: 123 days
Available cash: $101,530,610
